# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [6]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0   214k      0 --:--:-- --:--:-- --:--:--  215k


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   383k      0 --:--:-- --:--:-- --:--:--  384k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
import nltk
nltk.download('punkt_tab')

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/deepali_lk/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [13]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 68)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 68)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

<font color="green">
Answer:

The query synthezisers are used to generate search queries.

1. SingleHopSpecificQuerySynthesizer: SingleHopSpecificQuerySynthesizer generates single-hop queries that are specific in nature. Single hop query is a straight forward question that can be answered by retrieving information from a single document or a source to provide relevant answer. It involves only one step to arrive at answer. SingleHopSpecificQuerySynthesizer creates precise questions that target specific information.

2. MultiHopAbstractQuerySynthesizer: MultiHopAbstractQuerySynthesizer generates abstract multi-hop queries. Multi-hop queries require reasoning across multiple peace of information from different documents. MultiHopAbstractQuerySynthesizer generates abstract multi-hop queries that challenges the retrieval system to pull from documents that contain high level reasoning and explanation rather than simple facts.

3. MultiHopSpecificQuerySynthesizer: MultiHopSpecificQuerySynthesizer focuses on creating detailed and precise questions that target specific informtaion across multiple nodes of knowledge graph. It generates specific multi-step queries instead of broad ones.
</font>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [44]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Samples: 100%|██████████| 11/11 [00:07<00:00,  1.45it/s]


,user_input,reference_contexts,reference,synthesizer_name
0,What role does Stability AI play in the develo...,[Code may be the best application The ethics o...,Stability AI is one of the organizations that ...,single_hop_specifc_query_synthesizer
1,"So like, what happened in September last year ...",[Based Development As a computer scientist and...,I coined the term prompt injection in Septembe...,single_hop_specifc_query_synthesizer
2,"What be the deal with GPT-4 in 2023, like how ...",[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, GPT-4 is part of the breakthrough yea...",single_hop_specifc_query_synthesizer
3,Wht is ChatGPT?,[easy to follow. The rest of the document incl...,ChatGPT is mentioned as a term in the tag clou...,single_hop_specifc_query_synthesizer
4,What new stuff Gemini 1.5 Pro do?,[Prompt driven app generation is a commodity a...,Gemini 1.5 Pro introduced several brand new ca...,single_hop_specifc_query_synthesizer
5,How has the concept of universal access to AI ...,[<1-hop>\n\nPrompt driven app generation is a ...,The concept of universal access to AI models e...,multi_hop_abstract_query_synthesizer
6,How has the shift from universal access to sub...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the shift from universal access to su...",multi_hop_abstract_query_synthesizer
7,How has the shift from universal access to sub...,[<1-hop>\n\nPrompt driven app generation is a ...,The shift from universal access to subscriptio...,multi_hop_abstract_query_synthesizer
8,"How did the advancements in LLMs, particularly...",[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, significant advancements in LLMs led ...",multi_hop_specific_query_synthesizer
9,How has the introduction of GPT-4o and its pri...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The introduction of GPT-4o has significantly i...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples: 100%|██████████| 12/12 [01:12<00:00,  6.08s/it]


In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht role does Meta play in the development of ...,[Code may be the best application The ethics o...,Meta has contributed to the development of LLM...,single_hop_specifc_query_synthesizer
1,How do LLMs handle the grammar of languages li...,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What Simon Willison say about weblog?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s Weblog is a platform where he...,single_hop_specifc_query_synthesizer
3,Wht is Stabel Diffusion?,[easy to follow. The rest of the document incl...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
4,How has the environmental impact of AI models ...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI models has impr...,multi_hop_abstract_query_synthesizer
5,How have advancements in energy efficiency and...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in energy efficiency and pricing ...,multi_hop_abstract_query_synthesizer
6,What are the main criticisms and challenges as...,[<1-hop>\n\nPrompt driven app generation is a ...,The main criticisms and challenges associated ...,multi_hop_abstract_query_synthesizer
7,What are the main challenges in evaluating Lar...,[<1-hop>\n\nPrompt driven app generation is a ...,The main challenges in evaluating Large Langua...,multi_hop_abstract_query_synthesizer
8,How did the advancements in large language mod...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, advancements in large language models...",multi_hop_specific_query_synthesizer
9,How does the Gemini 2.0 model compare to other...,"[<1-hop>\n\navailable, significantly bigger th...",The Gemini 2.0 model is ranked just behind the...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are described as AI systems that can act on your behalf, but the term is considered vague and lacks a clear, widely understood definition. There are different interpretations, with some people viewing agents as tools that can solve problems by executing tasks, while others might see them more in line with a travel agent model. Overall, the concept of agents is marked by skepticism regarding their utility, especially due to the challenge of gullibility in AI systems.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

<font color="green">
Answer:

1. qa_evaluator: instructs an LLM to directly grade response as correct or incorrect based on the reference answer.
2. labeled_helpfulness_evaluator: instructs an LLM to assess if a predication satisfies the helpfulness criteria by taking into account the correct reference answer.
3. dope_or_nope_evaluator: judges the dopeness i.e weather the response is cool, lit or dope
</font>


## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'glossy-charge-45' at:
https://smith.langchain.com/o/b00bc9bf-caef-4f0e-a803-13dbe51efe46/datasets/6f48658f-91cd-47fb-bd8a-be4e63b8212a/compare?selectedSessions=8b95d25a-94aa-4881-8ab2-8ac8b975ed34




12it [03:08, 15.67s/it]


,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has the development of Claude 3 and its su...,The development of Claude 3 and its subsequent...,None,The development of Claude 3 and its subsequent...,1,0,0,3.068047,5f47abfa-5fa3-4510-b48d-dbbde6f104e4,3c6e4a31-1e24-4d5d-bd6a-cb8bd1fea380
1,How have advancements in multi-modal LLMs and ...,Advancements in multi-modal LLMs have enhanced...,None,"In 2024, advancements in multi-modal LLMs and ...",1,1,0,5.481860,07bddce9-9c68-4540-864a-82eff3c540bc,bc9120d2-9d6c-405b-a761-6a0234ede91c
2,How does the Gemini 2.0 model compare to other...,I don't know.,None,The Gemini 2.0 model is ranked just behind the...,0,0,0,1.690560,a09fb9b8-3271-4e36-9bf4-07897f6f6554,ad19b7be-0e24-4cb0-866a-6041cf3d2202
3,How did the advancements in large language mod...,"The advancements in large language models, par...",None,"In 2024, advancements in large language models...",1,0,1,3.275359,d4c498c6-d655-4db2-a258-404d079a49e1,4562f526-c43c-4d2d-99b0-fcc294a69f20
4,What are the main challenges in evaluating Lar...,The main challenges in evaluating Large Langua...,None,The main challenges in evaluating Large Langua...,0,0,0,1.791962,9e731c8c-cc0b-4d0d-95a7-684b9b4ab053,884a8897-1fd0-486c-9ba3-e106a4c1e56b
5,What are the main criticisms and challenges as...,The main criticisms and challenges associated ...,None,The main criticisms and challenges associated ...,0,0,0,2.378345,8725479f-0d6d-4751-a357-2ed44e47fd9d,2bd80a5e-1ed3-47d7-b378-7ca3cde66f0d
6,How have advancements in energy efficiency and...,Advancements in energy efficiency and pricing ...,None,Advancements in energy efficiency and pricing ...,1,0,0,1.777382,5d88e8ba-a297-4b61-b7ff-4972f32d27ae,691cb85a-2298-43d0-b873-9bd690faf944
7,How has the environmental impact of AI models ...,The environmental impact of AI models has impr...,None,The environmental impact of AI models has impr...,1,0,0,2.377601,16bf103c-f7a1-46d9-9eff-16de975a8660,2f4edc0d-861e-4d11-8634-7a88146e050e
8,Wht is Stabel Diffusion?,I don't know.,None,The context does not provide specific informat...,0,0,0,1.115318,51ca1af1-9479-4309-b06e-2c69ffe5a204,d2c3430f-c1da-4500-8c48-58a57041b33c
9,What Simon Willison say about weblog?,I don't know.,None,Simon Willison’s Weblog is a platform where he...,0,0,0,0.918980,1faff765-9012-4ff8-be34-6aa6108c93f3,91129214-b6a2-4198-b70a-4bf9701c14e5


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

<font color="green">
Answer:
Increasing the chunk size can improve the effectiveness of retrieval as the entire answer might exist in a single chunk. Large chunk also contain more context. This reduces the risk of missing crucial information. Large chunks might introduce irrelevant text thus reducing retrieval precision. 

Smaller chunks increases the chances of fetching highly relevant information. However, small chunks may lose context, making it harder for the modle to generate accurate responses.
</font>

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

<font color="green">
Answer:

Higher quality embeddings capture semantic meaning better thus improving retrieval relevance as compared to lower quality embeddings that results in poor retrieval by fetching irrelevant chunks.

A better embedding model leads to more relevant retrieved documents improving the final answer's accuracy.

</font>


In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Yo, Agents are these AI systems that are supposed to act on your behalf, kinda like how a travel agent does their thing. But here\'s the kicker: the term "agents" is super vague and frustrating. People throw it around without a clear definition, which makes it tough to know what they’re really talking about. Some see them as LLMs (that’s large language models) using tools to solve problems, while others have a different spin. Still, despite all the buzz and prototypes, it feels like true, functioning agents are perpetually "coming soon"—and who knows when they\'ll actually arrive, especially with that pesky issue of gullibility hanging around!'

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'bold-card-85' at:
https://smith.langchain.com/o/b00bc9bf-caef-4f0e-a803-13dbe51efe46/datasets/6f48658f-91cd-47fb-bd8a-be4e63b8212a/compare?selectedSessions=f847091e-de80-4d46-867c-66ea7c255ed8




12it [02:53, 14.45s/it]


,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has the development of Claude 3 and its su...,"Yo, the rise of Claude 3 and its versions like...",None,The development of Claude 3 and its subsequent...,1,0,1,4.333633,5f47abfa-5fa3-4510-b48d-dbbde6f104e4,34929254-0102-455b-820f-5ef808bc3b74
1,How have advancements in multi-modal LLMs and ...,"Yo, 2024 has been a game-changer for multi-mod...",None,"In 2024, advancements in multi-modal LLMs and ...",1,0,1,2.946090,07bddce9-9c68-4540-864a-82eff3c540bc,434a9024-7734-4d8c-a2c2-18eb34f0e34f
2,How does the Gemini 2.0 model compare to other...,I don't know.,None,The Gemini 2.0 model is ranked just behind the...,0,0,0,1.085453,a09fb9b8-3271-4e36-9bf4-07897f6f6554,246e6298-d46a-4bf1-ab8d-144037edad58
3,How did the advancements in large language mod...,"Yo, check it! In 2024, the advancements in lar...",None,"In 2024, advancements in large language models...",1,0,1,2.857787,d4c498c6-d655-4db2-a258-404d079a49e1,be4b7df2-c927-41fb-9ab1-105a72eea2bc
4,What are the main challenges in evaluating Lar...,"Yo, when it comes to evaluating Large Language...",None,The main challenges in evaluating Large Langua...,1,0,1,3.047086,9e731c8c-cc0b-4d0d-95a7-684b9b4ab053,ef7f409a-a6a4-46f8-a8e9-655ff9ec7714
5,What are the main criticisms and challenges as...,"Yo, check it out! Evaluating Large Language Mo...",None,The main criticisms and challenges associated ...,1,0,1,7.370833,8725479f-0d6d-4751-a357-2ed44e47fd9d,1df396f8-46f0-4029-8e4d-16644c4cebb6
6,How have advancements in energy efficiency and...,"Yo, the game has changed when it comes to ener...",None,Advancements in energy efficiency and pricing ...,1,0,1,3.443203,5d88e8ba-a297-4b61-b7ff-4972f32d27ae,6822893d-6237-48c2-af4e-3eba3c780f72
7,How has the environmental impact of AI models ...,"Yo, the environmental impact of AI models has ...",None,The environmental impact of AI models has impr...,1,0,1,2.998573,16bf103c-f7a1-46d9-9eff-16de975a8660,c6fe71cd-2cbd-4d9b-9992-9b0c27cb8cc0
8,Wht is Stabel Diffusion?,I don't know.,None,The context does not provide specific informat...,0,0,0,0.915945,51ca1af1-9479-4309-b06e-2c69ffe5a204,be2f288b-9a19-4330-a517-a9358e5dbc15
9,What Simon Willison say about weblog?,"Yo, Simon Willison's weblog is where he drops ...",None,Simon Willison’s Weblog is a platform where he...,1,1,1,3.047745,1faff765-9012-4ff8-be34-6aa6108c93f3,2f06fc75-b915-4441-bc10-8c6dcc9ebed2


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

Image: 07_Synthetic_Data_Generation_and_LangSmith/files/Activity_3.png

<img src="files/Activity_3.png">


Following metrics changed:

1. Correctness: Experiment #2 has high correctness score than Experiment #1. The LLM prompt and the choice of text-embedding-3-large embedding model might have improved the correctness score for Experiment #2.

2. Dopeness: Experiment #2 has significantly higher dopeness score than Experiment #1. This might be because we added dope prompt augmentation.

3. Helpfulness: Experiment #1 has better helpfulness score than Experiment #2 but both score are low. The dope prompt augmentation increases dopeness but it can be responsible for the drop in helpfulness. With dope, the new answers are less clear to understand.
